# Definições base do tabuleiro
-> tabuleiro deve seguir as imagens base

-> tabuleiro deve possuir pontos de transição que permitam o agente ir de um andar ao outro

-> tabuleiro precisa gerar as comidinhas

In [1]:
def criar_tabuleiro():
    tabuleiro_base = []
    # Cria uma matriz (lista de listas) com o elemento especificado
    for i in range(11):
        tabuleiro_base.append([])
        for j in range(11):
            if i == 0 or j == 0 or i == 10 or j == 10:
                #tabuleiro_base[i].append("0")
                tabuleiro_base[i].append("■")
            else:
                
                #tabuleiro_base[i].append('0')
                tabuleiro_base[i].append('1')
    return tabuleiro_base

def imprimir_tabuleiro(tabuleiro):
    for linha in tabuleiro:
        print(' '.join(str(celula) for celula in linha))

# Criar o tabuleiro 11x11 com '1' representando as comidas
tabuleiro_pacman = criar_tabuleiro()

# Exibir o tabuleiro
#imprimir_tabuleiro(tabuleiro_pacman)


In [2]:
def andar1():
    tabuleiro1 = criar_tabuleiro()
    paredes = [[2,1],[2,3],[2,5],[2,6],[2,7],[2,9],
               [3,3],
               [4,2], [4,3], [4,5],[4,7],[4,8],
               [5,5], [5,7],
               [6,4],[6,5], [6,7], [6,9],
               [7,7],
               [8,2], [8,3], [8,5],[8,6],[8,7], [8,9]]
    for parede in paredes:
        i, j = parede
        #tabuleiro1[i][j] = "0"
        tabuleiro1[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro1)
    tabuleiro1[1][1] = "P"
    tabuleiro1[9][9] = "F"
    return tabuleiro1


In [3]:
def andar2():
    tabuleiro2 = criar_tabuleiro()
    paredes = [[1,6],
    [2, 2], [2, 4], [2, 6], [2, 8],  # Terceira linha
    [3, 2], [3, 4], [3, 8], # Quarta linha
    [4,4], [4,5], [4,7],[4,8], # Quinta linha
    [5,2], [5,8], # Sexta linha
    [6, 2], [6,6],[6, 8],   # Sétima linha
    [7, 4], [7,6],  # Oitava linha
    [8,1],[8,2],[8,4],[8,6], [8,9],# Laterais da nona linha
    [9, 4]]  # Décima linha
    for parede in paredes:
        i, j = parede
        #tabuleiro2[i][j] = "0"
        tabuleiro2[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro2)
    return tabuleiro2


In [4]:
def andar3():
    tabuleiro3 = criar_tabuleiro()
    paredes = [[1,6],
    [2, 2], [2, 5], [2, 6],[2, 7], [2, 8],  # Terceira linha
    [3, 2], # Quarta linha
    [4,2], [4,4], [4,5],[4,6],[4,8], # Quinta linha
    [5,6], [5,8], # Sexta linha
    [6, 2], [6,4], [6,5], [6,6],   # Sétima linha
    [7, 2],  # Oitava linha
    [8,2],[8,3],[8,6],[8,7],[8,8],# Laterais da nona linha
    [9, 3]]  # Décima linha
    for parede in paredes:
        i, j = parede
        #tabuleiro3[i][j] = "0"
        tabuleiro3[i][j] = "■"
    #imprimir_tabuleiro(tabuleiro3)
    return tabuleiro3
    


# Gerando agentes


In [5]:
from GameState import GameState
import numpy as np
from numpy import Infinity
from IPython.display import clear_output
import time as time

In [6]:
gamestate = GameState([andar1(), andar2(), andar3()], True, [1,1,0], [9,9,0])

In [7]:
def MinMax(state:list, depth:int, turn:bool, moves:list):
    best = []
    if turn:
        best = [-1,-1,-1, Infinity]
    else:
        best = [-1, -1, -1, -Infinity]
    
    if depth == 0 or gamestate.is_terminal():
         score = gamestate.score(gamestate.pos_max if gamestate.turn_Max else gamestate.pos_min, best)
         return [-1, -1, -1, score]

    
    for i in moves:
        if turn:
            state = gamestate.get_new_state(gamestate.pos_max,i)
            #print(state.get_possible_moves())
            score = MinMax(state, depth-1, not turn, state.get_possible_moves())
            score[0], score[1], score[2] = i[0], i[1], i[2]
            if score[3] > best[3]:
                best = score
            print(best)
        else:
            state = gamestate.get_new_state(gamestate.pos_min,i)
            #print(state.get_possible_moves())
            score = MinMax(state, depth-1, not turn, state.get_possible_moves())
            score[0], score[1], score[2] = i[0], i[1], i[2]
            if score[3] > best[3]:
                best = score
            print(best)
    return best

In [8]:
def hill_climb(gamestate: GameState, pacman_pos, ghost_pos):
		ghost_pos = gamestate.pos_min
		pacman_pos = gamestate.pos_max
		distancia_original = gamestate.straight_distance()
		moves = gamestate.get_possible_moves()
		lista_movimentos = []

		melhorou = True
		while melhorou:
			melhorou = False
			melhor_distancia = distancia_original
			melhor_movimento = None 

			for move in moves:
				ghost_new_pos = [ghost_pos[0] + move[0], ghost_pos[1] + move[1]]
				nova_distancia = gamestate.straight_distance()

				if (nova_distancia < melhor_distancia):
					melhor_movimento = move
					melhor_distancia = nova_distancia
					melhorou = True
			
				if melhor_movimento:
					ghost_pos[0] = ghost_pos[0] + melhor_movimento[0]
					ghost_pos[1] = ghost_pos[1] + melhor_movimento[1]
					lista_movimentos.append(melhor_movimento)
					distancia_original = melhor_distancia
		
		return lista_movimentos


In [9]:
def movimento_jogador(gamestate):
    continuar = True
    while continuar:
        escolha = input("Digite o movimento desejado (cima, baixo, esquerda, direita, subir ou descer)")
        escolha = escolha.lower()
        pos = gamestate.pos_max
        if escolha == "cima" and gamestate.board_state[pos[2]][pos[0]-1][pos[1]] != "■":
                pos = [pos[0]-1,pos[1], pos[2]]
                continuar = False
                return pos
        elif escolha == "baixo" and gamestate.board_state[pos[2]][pos[0]+1][pos[1]] != "■":
                pos = [pos[0]+1,pos[1], pos[2]]
                continuar = False
                return pos
        elif escolha == "esquerda" and gamestate.board_state[pos[2]][pos[0]][pos[1]-1] != "■":
                pos = [pos[0], pos[1]-1, pos[2]]
                continuar = False
                return pos
        elif escolha == "direita" and gamestate.board_state[pos[2]][pos[0]][pos[1]+1] != "■":
                pos = [pos[0],pos[1]+1,pos[2]]
                continuar = False
                return pos
        elif escolha == "subir" and gamestate.board_state[pos[2]+1][pos[0]][pos[1]] != "■":
                pos = [pos[0],pos[1],pos[2]+1]
                continuar = False
                return pos
        elif escolha == "descer" and gamestate.board_state[pos[2]-1][pos[0]-1][pos[1]] != "■":
                pos = [pos[0],pos[1]-1,pos[2]-1]
                continuar = False
                return pos
        else:
                print("Movimento bloqueado, tente outra rota")

In [10]:
def iniciarJogo(jogador_inicial: bool, ia_escolhida):
    gamestate.turn_Max = jogador_inicial
    while not gamestate.is_terminal():
        if ia_escolhida == 'min-max':
            if gamestate.turn_Max:
                gamestate.get_new_state(gamestate.pos_max, movimento_jogador(gamestate))
                gamestate.turn_Max = False
            else:
                movimento_escolhido = MinMax(gamestate.board_state, gamestate.get_depth(), gamestate.turn_Max, gamestate.get_possible_moves())
                gamestate.get_new_state(gamestate.pos_min, movimento_escolhido)
                gamestate.turn_Max = True
            if movimento_escolhido[3] == 0:
                print(gamestate.board_state[0])
            elif movimento_escolhido[3] == 1:
                print(gamestate.board_state[1])
            else:
                print(gamestate.board_state[2])
        elif ia_escolhida == 'hill_climb':
            if gamestate.turn_Max:
                gamestate.get_new_state(gamestate.pos_max, movimento_jogador(gamestate))
                gamestate.turn_Max = False
            else: movimento_escolhido = hill_climb(gamestate, gamestate.pos_max, gamestate.pos_min)

In [11]:
iniciarJogo(True, 'min-max')

KeyboardInterrupt: 